In [1]:
!pip install xarray netCDF4 pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.4 MB/s eta 0:00:00


In [2]:
import xarray as xr
import pandas as pd


In [3]:
# Descarga del archivo mensual global
!wget https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_monthly/netcdf/chirps-v2.0.monthly.nc


--2025-11-18 00:13:12--  https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_monthly/netcdf/chirps-v2.0.monthly.nc
Resolving data.chc.ucsb.edu (data.chc.ucsb.edu)... 128.111.100.31
Connecting to data.chc.ucsb.edu (data.chc.ucsb.edu)|128.111.100.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7547536807 (7.0G) [application/octet-stream]
Saving to: ‘chirps-v2.0.monthly.nc’

chirps-v2.0.monthly 100%[===================>]   7.03G  29.5MB/s    in 3m 27s  

2025-11-18 00:16:39 (34.8 MB/s) - ‘chirps-v2.0.monthly.nc’ saved [7547536807/7547536807]



In [4]:
ds = xr.open_dataset("chirps-v2.0.monthly.nc")
ds


<xarray.Dataset> Size: 31GB
Dimensions:    (time: 537, latitude: 2000, longitude: 7200)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1981-01-01 1981-02-01 ... 2025-09-01
  * latitude   (latitude) float32 8kB -49.97 -49.92 -49.88 ... 49.87 49.92 49.97
  * longitude  (longitude) float32 29kB -180.0 -179.9 -179.9 ... 179.9 180.0
Data variables:
    precip     (time, latitude, longitude) float32 31GB ...
Attributes: (12/15)
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    history:           created by Climate Hazards Group
    version:           Version 2.0
    date_created:      2025-10-15
    creator_name:      Pete Peterson
    ...                ...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    comments:           time variable denotes the first day of the given month.
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CH...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ

In [5]:
print("Latitudes:", ds.latitude.values.min(), "→", ds.latitude.values.max())
print("Longitudes:", ds.longitude.values.min(), "→", ds.longitude.values.max())


Latitudes: -49.975 → 49.975
Longitudes: -179.975 → 179.975


In [6]:
# -----------------------------
#  EJE CAFETERO
# -----------------------------

# Caldas
lat_min_caldas, lat_max_caldas = 4.7, 6.1
lon_min_caldas, lon_max_caldas = -75.9, -74.8

# Quindío
lat_min_quindio, lat_max_quindio = 4.2, 4.9
lon_min_quindio, lon_max_quindio = -75.9, -75.3

# Risaralda
lat_min_risaralda, lat_max_risaralda = 4.6, 5.4
lon_min_risaralda, lon_max_risaralda = -76.1, -75.3

# -----------------------------
#  ZONA SUR
# -----------------------------

# Cauca
lat_min_cauca, lat_max_cauca = 1.0, 3.3
lon_min_cauca, lon_max_cauca = -77.6, -75.5

# Nariño
lat_min_narino, lat_max_narino = 0.5, 2.3
lon_min_narino, lon_max_narino = -78.3, -76.5


In [7]:
pr = ds['precip']

# -----------------------------
#  EJE CAFETERO
# -----------------------------

# Caldas
pr_caldas = pr.sel(
    latitude=slice(lat_min_caldas, lat_max_caldas),
    longitude=slice(lon_min_caldas, lon_max_caldas)
)

# Quindío
pr_quindio = pr.sel(
    latitude=slice(lat_min_quindio, lat_max_quindio),
    longitude=slice(lon_min_quindio, lon_max_quindio)
)

# Risaralda
pr_risaralda = pr.sel(
    latitude=slice(lat_min_risaralda, lat_max_risaralda),
    longitude=slice(lon_min_risaralda, lon_max_risaralda)
)

# -----------------------------
#  ZONA SUR
# -----------------------------

# Cauca
pr_cauca = pr.sel(
    latitude=slice(lat_min_cauca, lat_max_cauca),
    longitude=slice(lon_min_cauca, lon_max_cauca)
)

# Nariño
pr_narino = pr.sel(
    latitude=slice(lat_min_narino, lat_max_narino),
    longitude=slice(lon_min_narino, lon_max_narino)
)


In [8]:
print("Caldas shape:", pr_caldas.shape)
print("Quindío shape:", pr_quindio.shape)
print("Risaralda shape:", pr_risaralda.shape)
print("Cauca shape:", pr_cauca.shape)
print("Nariño shape:", pr_narino.shape)


Caldas shape: (537, 28, 22)
Quindío shape: (537, 14, 12)
Risaralda shape: (537, 16, 16)
Cauca shape: (537, 46, 42)
Nariño shape: (537, 36, 36)


In [9]:
import pandas as pd

# -----------------------------
#  PROMEDIOS ESPACIALES
# -----------------------------

# Caldas
ts_caldas = pr_caldas.mean(dim=["latitude", "longitude"]).to_series()
ts_caldas.name = "precip_caldas_mm"

# Quindío
ts_quindio = pr_quindio.mean(dim=["latitude", "longitude"]).to_series()
ts_quindio.name = "precip_quindio_mm"

# Risaralda
ts_risaralda = pr_risaralda.mean(dim=["latitude", "longitude"]).to_series()
ts_risaralda.name = "precip_risaralda_mm"

# Cauca
ts_cauca = pr_cauca.mean(dim=["latitude", "longitude"]).to_series()
ts_cauca.name = "precip_cauca_mm"

# Nariño
ts_narino = pr_narino.mean(dim=["latitude", "longitude"]).to_series()
ts_narino.name = "precip_narino_mm"

# -----------------------------
#  DATAFRAME CONSOLIDADO
# -----------------------------

df_chirps = pd.concat([
    ts_caldas,
    ts_quindio,
    ts_risaralda,
    ts_cauca,
    ts_narino
], axis=1)

df_chirps.head()


,precip_caldas_mm,precip_quindio_mm,precip_risaralda_mm,precip_cauca_mm,precip_narino_mm
time,,,,,
1981-01-01,76.724571,53.704510,58.670631,125.502731,161.339127
1981-02-01,160.978897,144.632324,146.745209,204.025467,215.816605
1981-03-01,192.058456,204.694901,183.591583,240.549194,296.788147
1981-04-01,337.048096,306.992340,322.701599,327.348755,313.295227
1981-05-01,402.448303,305.212463,324.900574,336.320709,283.058075


In [10]:
# Exportar el DataFrame consolidado a CSV
df_chirps.to_csv("precipitacion_departamentos_chirps.csv", index=True)

print("Archivo generado: precipitacion_departamentos_chirps.csv")


Archivo generado: precipitacion_departamentos_chirps.csv
